###  <font color='red'>  Important: </font> Before proceeding, run the following cell to check for code updates.

In [ ]:
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

# Brain Tumor Segmentation (BraTS) with OpenVINO™

In this code example, we apply the U-Net architecture to segment brain tumors from raw MRI scans. With relatively little data we are able to train a U-Net model to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for our model is about 0.82-0.88.  Menze et al. [reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made. The below MRI brain scans highlight brain tumor matter segmented using deep learning. 

<img src="images/figure1.png">

## Demonstration objectives
* Healthcare use-case demo
* **Model Optimizer** in action
* U-Net based segmentation on edge hardware
* Inferencing with OpenVINO™ Inference Engine
* Running inference across CPU, integrated GPU, VPU, and FPGA and comparing throughput and latency

## What is U-Net?
Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597.pdf0) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the “U” shape.  However, in contrast to the autoencoder, U-Net predicts a pixelwise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

<img src="images/unet.png">

## Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel

The Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.

###  Generate FP16 and FP32 Optimized Model
Now, let's convert the model to the optimized model using the model optimizer. This model works best on most hardware, including CPUs.

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py \
            --input_model /data/Healthcare_app/data/saved_model_frozen.pb  \
            --input_shape=[1,144,144,4] \
            --data_type FP16  \
            --output_dir output/IR_models/FP16  \
            --model_name saved_model \
            --silent

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py \
            --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
            --input_shape=[1,144,144,4] \
            --data_type FP32  \
            --output_dir output/IR_models/FP32  \
            --model_name saved_model \
            --silent

In [ ]:
# Install required Packages
import os
import sys
try:
    import psutil
except:
    print("psutil not installed")
    os.system("{} -m pip install psutil".format(sys.executable))

try:
    import keras as K
except:
    os.system("{} -m pip install keras".format(sys.executable))

## Import libraries and configuration file for widget management.

In [ ]:
import json
import qarpo

with open('job_config.json') as json_file:
    data = json.load(json_file)

## Display available options for Intel® Edge compute hardware and acceleration, results and performance summary.

In [ ]:
interface = qarpo.Interface(data)
interface.displayUI()